In [8]:
#imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np

In [9]:
#gpu config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


In [10]:
#hyperparams
learning_rate = 0.01
batch_size = 128
num_epochs = 30

#load data
transform = transforms.Compose([transforms.ToTensor()])
train_set = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# Define the model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(3*32*32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
#initialize
model = MLP().to(device)
writer = SummaryWriter()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
#Training

#train net
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        
        # Send data to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward pass and compute loss
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute running train loss
        train_loss += loss.item()
        
    #validation
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    valid_loss = valid_loss/len(test_loader.dataset)
    accuracy = 100 * correct / total

    writer.add_scalar('Training Loss', train_loss / len(train_loader), epoch)
    writer.add_scalar('Validation Loss', valid_loss / len(test_loader), epoch)
    
    print(f"Epoch {epoch+1}/{30}, Training Loss: {train_loss:.6f}, Validation Loss: {valid_loss:.6f}, Accuracy: {accuracy:.2f}")
writer.close()

Epoch 1/30, Training Loss: 875.634878, Validation Loss: 2.142435, Accuracy: 25.39
Epoch 2/30, Training Loss: 802.100633, Validation Loss: 1.988378, Accuracy: 27.88
Epoch 3/30, Training Loss: 759.226688, Validation Loss: 1.913028, Accuracy: 30.99
Epoch 4/30, Training Loss: 735.244952, Validation Loss: 1.857433, Accuracy: 33.42
Epoch 5/30, Training Loss: 718.529811, Validation Loss: 1.818353, Accuracy: 35.84
Epoch 6/30, Training Loss: 704.211717, Validation Loss: 1.779263, Accuracy: 36.78
Epoch 7/30, Training Loss: 691.667022, Validation Loss: 1.748069, Accuracy: 38.40
Epoch 8/30, Training Loss: 679.916935, Validation Loss: 1.721431, Accuracy: 38.47
Epoch 9/30, Training Loss: 669.644006, Validation Loss: 1.702396, Accuracy: 39.70
Epoch 10/30, Training Loss: 660.427999, Validation Loss: 1.673836, Accuracy: 40.75
Epoch 11/30, Training Loss: 651.950337, Validation Loss: 1.661166, Accuracy: 41.22
Epoch 12/30, Training Loss: 644.256432, Validation Loss: 1.650628, Accuracy: 41.28
Epoch 13/30, 